In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/데이터 과학/rawg_games_data.csv")

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881069 entries, 0 to 881068
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           881069 non-null  int64  
 1   name         881066 non-null  object 
 2   released     852051 non-null  object 
 3   rating       881069 non-null  float64
 4   genres       667676 non-null  object 
 5   platforms    876907 non-null  object 
 6   tags         782916 non-null  object 
 7   metacritic   7134 non-null    float64
 8   developers   870442 non-null  object 
 9   publishers   211931 non-null  object 
 10  playtime     881069 non-null  int64  
 11  description  797536 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 80.7+ MB


In [27]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881069 entries, 0 to 881068
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           881069 non-null  int64  
 1   name         881066 non-null  object 
 2   released     852051 non-null  object 
 3   rating       881069 non-null  float64
 4   genres       667676 non-null  object 
 5   platforms    876907 non-null  object 
 6   tags         782916 non-null  object 
 7   metacritic   7134 non-null    float64
 8   developers   870442 non-null  object 
 9   publishers   211931 non-null  object 
 10  playtime     881069 non-null  int64  
 11  description  797536 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 80.7+ MB


None

In [28]:
missing_values = df.isnull().sum()
display(missing_values)

,0
id,0
name,3
released,29018
rating,0
genres,213393
platforms,4162
tags,98153
metacritic,873935
developers,10627
publishers,669138


In [29]:
# 1. metacritic과 pubilsher 컬럼은 모델 학습에서 제외
df_processed = df.drop(['metacritic', 'publishers'], axis=1)

# 2. 'name' 또는 'released' 컬럼에 isnull인 행은 삭제 (핵심 정보 유지)
df_processed.dropna(subset=['name', 'released'], inplace=True)


# 3. developers와 description 컬럼도 학습에서 제외
df_processed = df_processed.drop(['developers', 'description'], axis=1)

# Note: 'genres', 'platforms', 'tags' 컬럼의 결측값은 그대로 유지하여 MultiLabelBinarizer가 처리하도록 합니다.


# 4. 이렇게 필터링된 데이터셋의 개수 보여줘
print(f"필터링 후 데이터셋의 개수 (행 수): {df_processed.shape[0]}")
display(df_processed.head())
display(df_processed.info()) # Add info to see if genres, platforms, tags are still there

필터링 후 데이터셋의 개수 (행 수): 852048


,id,name,released,rating,genres,platforms,tags,playtime
0,25097,The Legend of Zelda: Ocarina of Time,1998-11-21,4.38,"Action, Adventure, RPG","Nintendo Switch, Nintendo 64",Singleplayer,7
1,54751,Soulcalibur,1998-07-30,4.37,"Action, Fighting","Xbox One, iOS, Xbox 360, Android, Dreamcast",2 players,6
2,407559,Soulcalibur (1998),1998-07-30,0.00,Fighting,"Dreamcast, Xbox 360",NaN,0
3,27036,Super Mario Galaxy 2,2010-05-23,4.34,Platformer,"Wii, Wii U","collect, Solo, galaxy, light, castle, Gravity,...",18
4,324997,Baldur's Gate III,2023-08-03,4.47,"Adventure, RPG, Strategy","macOS, PC, Xbox Series S/X, PlayStation 5","Singleplayer, Steam Achievements, Multiplayer,...",19


<class 'pandas.core.frame.DataFrame'>
Index: 852048 entries, 0 to 881068
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         852048 non-null  int64  
 1   name       852048 non-null  object 
 2   released   852048 non-null  object 
 3   rating     852048 non-null  float64
 4   genres     642481 non-null  object 
 5   platforms  848552 non-null  object 
 6   tags       763531 non-null  object 
 7   playtime   852048 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 58.5+ MB


None

In [30]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convert 'released' column to datetime
# Check if 'released' column exists before converting
if 'released' in df_processed.columns:
    df_processed['released'] = pd.to_datetime(df_processed['released'])
else:
    print("Warning: 'released' column not found in df_processed.")


# Reset index after dropping rows to ensure a clean 0-based index for joining
# This was already done in cell 307b5233, but let's ensure a clean index before encoding/concatenating
df_processed.reset_index(drop=True, inplace=True)

# Function to apply MultiLabelBinarizer with frequency filtering
def encode_multilabel_filtered(df, column, threshold=10):
    if column not in df.columns:
        print(f"Warning: Column '{column}' not found in DataFrame. Skipping encoding.")
        return pd.DataFrame(index=df.index) # Return empty DataFrame with matching index

    mlb = MultiLabelBinarizer()
    # Safely split and handle potential NaN values after dropping rows
    split_values = df[column].str.split(', ').dropna()

    # If after dropping NaNs, there are no values left, return empty DataFrame
    if split_values.empty:
        print(f"Warning: No non-null values in column '{column}' after splitting. Skipping encoding.")
        return pd.DataFrame(index=df.index)

    # Calculate frequencies of each unique value
    all_values = [item for sublist in split_values for item in sublist]
    value_counts = pd.Series(all_values).value_counts()

    # Identify values to keep (frequency >= threshold)
    values_to_keep = value_counts[value_counts >= threshold].index.tolist()

    # Filter the split values to keep only values above the threshold
    filtered_split_values = split_values.apply(lambda x: [item for item in x if item in values_to_keep])

    # Apply MultiLabelBinarizer to the filtered values
    # Ensure there are values left after filtering before fitting
    if filtered_split_values.empty:
         print(f"Warning: No values left in column '{column}' after frequency filtering. Skipping encoding.")
         return pd.DataFrame(index=df.index)

    mlb = MultiLabelBinarizer()
    encoded_data = mlb.fit_transform(filtered_split_values)

    # Align index after fit_transform - use the index of filtered_split_values
    encoded_df = pd.DataFrame(encoded_data, columns=mlb.classes_, index=filtered_split_values.index)
    return encoded_df

# Function to encode top N frequent multi-label features
def encode_top_n_multilabel(df, column, top_n=30): # Changed top_n to 30 here as requested
    if column not in df.columns:
        print(f"Warning: Column '{column}' not found in DataFrame. Skipping encoding.")
        return pd.DataFrame(index=df.index) # Return empty DataFrame with matching index

    # Safely split and handle potential NaN values after dropping rows
    split_values = df[column].str.split(', ').dropna()

    # If after dropping NaNs, there are no values left, return empty DataFrame
    if split_values.empty:
        print(f"Warning: No non-null values in column '{column}' after splitting. Skipping encoding.")
        return pd.DataFrame(index=df.index)

    # Calculate frequencies of each unique value
    all_values = [item for sublist in split_values for item in sublist]
    value_counts = pd.Series(all_values).value_counts()

    # Identify the top N frequent values
    # Ensure there are enough values to select top_n from
    if len(value_counts) == 0:
        print(f"Warning: No unique values found in column '{column}'. Skipping encoding.")
        return pd.DataFrame(index=df.index)

    top_n_values = value_counts.head(top_n).index.tolist()

    # Filter the split values to keep only top N values
    filtered_split_values = split_values.apply(lambda x: [item for item in x if item in top_n_values])

    # Apply MultiLabelBinarizer to the filtered values
    # Ensure there are values left after filtering before fitting
    if filtered_split_values.empty:
         print(f"Warning: No values left in column '{column}' after top N filtering. Skipping encoding.")
         return pd.DataFrame(index=df.index)

    mlb = MultiLabelBinarizer()
    encoded_data = mlb.fit_transform(filtered_split_values)

    # Align index after fit_transform - use the index of filtered_split_values
    encoded_df = pd.DataFrame(encoded_data, columns=mlb.classes_, index=filtered_split_values.index)
    return encoded_df


# Apply MultiLabelBinarizer to genres and platforms with frequency filtering
genres_encoded_df = encode_multilabel_filtered(df_processed, 'genres', threshold=30)
platforms_encoded_df = encode_multilabel_filtered(df_processed, 'platforms', threshold=30)

# Apply top N encoding to tags
tags_encoded_df = encode_top_n_multilabel(df_processed, 'tags', top_n=30)


# Rename overlapping columns in ALL encoded dataframes before joining
# Re-calculate original_columns_after_drop in case columns were dropped
original_columns_after_drop = [col for col in df_processed.columns if col not in ['genres', 'platforms', 'tags']]

# Rename overlapping columns in genres_encoded_df if it's not empty
if not genres_encoded_df.empty:
    overlapping_genres = list(set(original_columns_after_drop) & set(genres_encoded_df.columns))
    rename_map_genres = {col: f"{col}_genre" for col in overlapping_genres}
    genres_encoded_df.rename(columns=rename_map_genres, inplace=True)

# Rename overlapping columns in platforms_encoded_df if it's not empty
if not platforms_encoded_df.empty:
    overlapping_platforms = list(set(original_columns_after_drop) & set(platforms_encoded_df.columns))
    rename_map_platforms = {col: f"{col}_platform" for col in overlapping_platforms}
    platforms_encoded_df.rename(columns=rename_map_platforms, inplace=True)

# Rename overlapping columns in tags_encoded_df if it's not empty
if not tags_encoded_df.empty:
    overlapping_tags = list(set(original_columns_after_drop) & set(tags_encoded_df.columns))
    rename_map_tags = {col: f"{col}_tag" for col in overlapping_tags}
    tags_encoded_df.rename(columns=rename_map_tags, inplace=True)


# Drop the original multi-label columns if they still exist from df_processed
columns_to_drop_from_original = ['genres', 'platforms', 'tags']
df_processed_base = df_processed.drop(columns=[col for col in columns_to_drop_from_original if col in df_processed.columns]).copy()

# Ensure the base DataFrame has a clean index for merging
df_processed_base.reset_index(drop=True, inplace=True)


# Now, the merging will happen in the next cell.
# Display the head/info of the base and encoded dataframes to confirm
print("df_processed_base (Original columns after dropping multi-label):")
display(df_processed_base.head())
display(df_processed_base.info(memory_usage='deep'))

print("\ngenres_encoded_df:")
display(genres_encoded_df.head())
display(genres_encoded_df.info(memory_usage='deep'))

print("\nplatforms_encoded_df:")
display(platforms_encoded_df.head())
display(platforms_encoded_df.info(memory_usage='deep'))

print("\ntags_encoded_df:")
display(tags_encoded_df.head())
display(tags_encoded_df.info(memory_usage='deep'))

# The merging step will be in the next cell.

df_processed_base (Original columns after dropping multi-label):


,id,name,released,rating,playtime
0,25097,The Legend of Zelda: Ocarina of Time,1998-11-21,4.38,7
1,54751,Soulcalibur,1998-07-30,4.37,6
2,407559,Soulcalibur (1998),1998-07-30,0.00,0
3,27036,Super Mario Galaxy 2,2010-05-23,4.34,18
4,324997,Baldur's Gate III,2023-08-03,4.47,19


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852048 entries, 0 to 852047
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        852048 non-null  int64         
 1   name      852048 non-null  object        
 2   released  852048 non-null  datetime64[ns]
 3   rating    852048 non-null  float64       
 4   playtime  852048 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 88.9 MB


None


genres_encoded_df:


...

None


tags_encoded_df:


,2D,3D,Action-Adventure,Atmospheric,Colorful,Cute,Exploration,Fantasy,First-Person,Full controller support,...,Short,Singleplayer,Space,Steam Achievements,Steam Cloud,Story Rich,Top-Down,Unity,fun,jam
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,1,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
Index: 763531 entries, 0 to 852047
Data columns (total 30 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   2D                       763531 non-null  int64
 1   3D                       763531 non-null  int64
 2   Action-Adventure         763531 non-null  int64
 3   Atmospheric              763531 non-null  int64
 4   Colorful                 763531 non-null  int64
 5   Cute                     763531 non-null  int64
 6   Exploration              763531 non-null  int64
 7   Fantasy                  763531 non-null  int64
 8   First-Person             763531 non-null  int64
 9   Full controller support  763531 non-null  int64
 10  Funny                    763531 non-null  int64
 11  Horror                   763531 non-null  int64
 12  Multiplayer              763531 non-null  int64
 13  Music                    763531 non-null  int64
 14  Physics                  763531 non-null 

None

In [31]:
# Merge the base DataFrame with the encoded DataFrames using their index
# Start with the base DataFrame
df_processed = df_processed_base.copy()

# Merge genres_encoded_df if it's not empty
if not genres_encoded_df.empty:
    df_processed = df_processed.merge(genres_encoded_df, left_index=True, right_index=True, how='left')

# Merge platforms_encoded_df if it's not empty
if not platforms_encoded_df.empty:
     df_processed = df_processed.merge(platforms_encoded_df, left_index=True, right_index=True, how='left')

# Merge tags_encoded_df if it's not empty
if not tags_encoded_df.empty:
    df_processed = df_processed.merge(tags_encoded_df, left_index=True, right_index=True, how='left')


# Display the head and info of the final df_processed
print("Final df_processed after merging encoded features:")
display(df_processed.head())
display(df_processed.info(memory_usage='deep'))

# Check for duplicate index in the final df_processed
if df_processed.index.has_duplicates:
    print("\nWarning: Duplicate index found in final df_processed after merging.")
    print("Duplicate index values:", df_processed.index[df_processed.index.duplicated()].unique().tolist())
else:
    print("\nNo duplicate index found in final df_processed after merging.")

Final df_processed after merging encoded features:


,id,name,released,rating,playtime,Action,Adventure,Arcade,Board Games,Card,...,Short,Singleplayer,Space,Steam Achievements,Steam Cloud,Story Rich,Top-Down,Unity,fun,jam
0,25097,The Legend of Zelda: Ocarina of Time,1998-11-21,4.38,7,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,54751,Soulcalibur,1998-07-30,4.37,6,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,407559,Soulcalibur (1998),1998-07-30,0.00,0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27036,Super Mario Galaxy 2,2010-05-23,4.34,18,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,324997,Baldur's Gate III,2023-08-03,4.47,19,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852048 entries, 0 to 852047
Columns: 103 entries, id to jam
dtypes: datetime64[ns](1), float64(99), int64(2), object(1)
memory usage: 726.0 MB


None


No duplicate index found in final df_processed after merging.


In [32]:
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler
scaler = StandardScaler()

# Select numerical columns to scale
numerical_cols = ['rating', 'playtime']

# Apply StandardScaler to the selected columns
df_processed[numerical_cols] = scaler.fit_transform(df_processed[numerical_cols])

# Display the first few rows of the scaled data
display(df_processed.head())

,id,name,released,rating,playtime,Action,Adventure,Arcade,Board Games,Card,...,Short,Singleplayer,Space,Steam Achievements,Steam Cloud,Story Rich,Top-Down,Unity,fun,jam
0,25097,The Legend of Zelda: Ocarina of Time,1998-11-21,8.819271,1.455220,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,54751,Soulcalibur,1998-07-30,8.798813,1.242334,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,407559,Soulcalibur (1998),1998-07-30,-0.141391,-0.034983,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27036,Super Mario Galaxy 2,2010-05-23,8.737439,3.796968,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,324997,Baldur's Gate III,2023-08-03,9.003394,4.009854,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [33]:
# Identify the one-hot encoded genre columns
# These are the columns added by MultiLabelBinarizer for genres in cell a2260c50
# We need to exclude the original columns and the platforms/tags encoded columns
# Assuming the genre column names are the classes from MultiLabelBinarizer
# Let's get the genre column names from the genres_encoded_df before concatenation, or by checking column names in df_processed after concatenation

# A more robust way is to get the columns that were added by genres_encoded_df
# We need to check the columns in df_processed after cell a2260c50
# Let's assume the columns added by genres_encoded_df are the ones that are not in the original df_processed (before encoding)
# This requires knowing the original columns before encoding

# Let's list the columns that were added by genres_encoded_df in cell a2260c50
# Looking at the output of cell a2260c50, the genre columns are directly named after the genres.
# We can identify them by checking which columns were NOT in df_processed before cell a2260c50,
# excluding the platforms and tags columns which were also added.
# A simpler approach for now is to assume the genre column names are the ones we expect based on the encoding (Action, Adventure, etc.)
# But it's safer to use the columns from the genres_encoded_df variable if it's available and accurate.

# Let's re-run cell a2260c50 to ensure genres_encoded_df is available and accurate
# Or, we can identify the genre columns based on the known genre list or a pattern.
# Let's try to identify them by looking at the columns in df_processed that are likely genres based on the encoding process.

# Assuming genre columns are the ones that were created from genres_encoded_df
# We need to be careful as platforms and tags were also added.
# Let's try to get the column names from the mlb.classes_ if we can access it from the function,
# or rely on the genres_encoded_df variable.

# Let's check if genres_encoded_df is in the environment
if 'genres_encoded_df' in locals():
    genre_cols = genres_encoded_df.columns.tolist()
    print(f"Identified {len(genre_cols)} genre columns for interaction.")
else:
    # Fallback: Try to identify genre columns based on a potential naming pattern or a known list
    # This is less robust if the encoding process changed or column names are unexpected
    print("Warning: 'genres_encoded_df' not found. Attempting to identify genre columns based on names.")
    # Assuming genre columns are title-cased and not original columns like 'id', 'name', etc.
    # This is a heuristic and might not be perfect.
    original_cols_before_encoding = ['id', 'name', 'released', 'rating', 'playtime'] # Add other original columns if any were kept
    # Let's get columns that are likely one-hot encoded based on their presence in the df_processed after encoding
    # and excluding the original columns and potentially platforms/tags if they have a different pattern
    potential_encoded_cols = [col for col in df_processed.columns if col not in original_cols_before_encoding]

    # This fallback is tricky without a clear pattern or access to the original encoded df columns.
    # A better approach is to ensure the previous cell that created genres_encoded_df is run.
    # Let's add a clear instruction to run the previous cell if genres_encoded_df is missing.
    print("Please ensure cell a2260c50 has been executed successfully before running this cell.")
    genre_cols = [] # Initialize as empty to prevent errors

# Ensure playtime is in the dataframe and is numerical (it should be after scaling)
if 'playtime' not in df_processed.columns:
    print("Error: 'playtime' column not found in df_processed.")
    playtime_col_exists = False
else:
     playtime_col_exists = True

# --- Add check for duplicate index here ---
if df_processed.index.has_duplicates:
    print("\nError: Duplicate index found in df_processed. Cannot create interaction terms.")
    print("Duplicate index values:", df_processed.index[df_processed.index.duplicated()].unique().tolist())
    # Decide how to proceed: maybe reset index or investigate source of duplicates
    # For now, we will stop here and report the issue.
else:
    print("\nNo duplicate index found in df_processed. Proceeding with interaction term creation.")

    # --- Explicitly reset index before creating interaction terms ---
    df_processed.reset_index(drop=True, inplace=True)
    print("Index reset before creating interaction terms.")


    if genre_cols and playtime_col_exists:
        # Create interaction terms
        for genre_col in genre_cols:
            new_col_name = f"playtime_x_{genre_col}"
            # Check if the genre column exists in the current df_processed before multiplying
            if genre_col in df_processed.columns:
                # Explicitly align the series before multiplication
                playtime_series = df_processed['playtime']
                genre_series = df_processed[genre_col]
                aligned_playtime, aligned_genre = playtime_series.align(genre_series, fill_value=0) # Use fill_value=0 for element-wise multiplication

                df_processed[new_col_name] = aligned_playtime * aligned_genre
                # print(f"Created interaction term: {new_col_name}")
            else:
                print(f"Warning: Genre column '{genre_col}' not found in df_processed. Skipping interaction term.")


        print(f"\nCreated {len([col for col in df_processed.columns if 'playtime_x_' in col])} interaction term features.")
        display(df_processed.head())
        display(df_processed.info(memory_usage='deep'))

    else:
        print("\nSkipping interaction term creation due to missing genre or playtime columns.")

Identified 19 genre columns for interaction.

No duplicate index found in df_processed. Proceeding with interaction term creation.
Index reset before creating interaction terms.

Created 18 interaction term features.


,id,name,released,rating,playtime,Action,Adventure,Arcade,Board Games,Card,...,playtime_x_Fighting,playtime_x_Indie,playtime_x_Massively Multiplayer,playtime_x_Platformer,playtime_x_Puzzle,playtime_x_Racing,playtime_x_Shooter,playtime_x_Simulation,playtime_x_Sports,playtime_x_Strategy
0,25097,The Legend of Zelda: Ocarina of Time,1998-11-21,8.819271,1.455220,1.0,1.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,54751,Soulcalibur,1998-07-30,8.798813,1.242334,1.0,0.0,0.0,0.0,0.0,...,1.242334,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,407559,Soulcalibur (1998),1998-07-30,-0.141391,-0.034983,0.0,0.0,0.0,0.0,0.0,...,-0.034983,-0.0,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000000
3,27036,Super Mario Galaxy 2,2010-05-23,8.737439,3.796968,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,3.796968,0.0,0.0,0.0,0.0,0.0,0.000000
4,324997,Baldur's Gate III,2023-08-03,9.003394,4.009854,0.0,1.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,4.009854


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852048 entries, 0 to 852047
Columns: 121 entries, id to playtime_x_Strategy
dtypes: datetime64[ns](1), float64(118), int64(1), object(1)
memory usage: 843.0 MB


None

In [34]:
# Create the target variable 'Hit' based on the specified complex criteria
# Assume rating and playtime have already been scaled

# Calculate the scaled threshold for rating >= 3.5
original_rating_threshold = 3.5
scaled_rating_threshold = (original_rating_threshold - scaler.mean_[0]) / scaler.scale_[0]

# Calculate the scaled value for rating == 0.0
scaled_zero_rating = (0.0 - scaler.mean_[0]) / scaler.scale_[0]


# Identify the interaction term columns (playtime_x_genre)
interaction_cols = [col for col in df_processed.columns if 'playtime_x_' in col]

# Define the top percentage threshold for interaction terms
interaction_percentile_threshold = 70 # 70th percentile corresponds to top 30%

# Create a boolean Series for the first part of the OR condition: rating >= 3.5
criterion_rating_high = (df_processed['rating'] >= scaled_rating_threshold)

# Create a boolean Series for the rating != 0.0 part of the second condition
criterion_rating_not_zero = (df_processed['rating'] != scaled_zero_rating)

# Create boolean masks for the top 30% criteria for each interaction term
interaction_top_30_masks = []
if interaction_cols: # Ensure there are interaction columns
    for col in interaction_cols:
        # Check if the column exists before calculating percentile
        if col in df_processed.columns:
            # Calculate the threshold for the top 30% for this specific interaction term
            # Use only non-NaN values for percentile calculation
            col_values = df_processed[col].dropna()
            if not col_values.empty:
                threshold_value = col_values.quantile(interaction_percentile_threshold / 100.0)
                # Create a boolean mask for this column (handle NaNs by setting to False)
                mask = (df_processed[col] >= threshold_value).fillna(False)
                interaction_top_30_masks.append(mask)
            else:
                 print(f"Warning: Interaction column '{col}' has no non-NaN values for percentile calculation. Skipping.")
        else:
             print(f"Warning: Interaction column '{col}' not found in df_processed. Skipping percentile calculation.")


# Combine the interaction term masks with logical OR
# If no valid interaction masks were created, this part of the criterion will be False
criterion_interaction_any_top_30 = pd.Series(False, index=df_processed.index) # Default to False
if interaction_top_30_masks:
     # Use pd.concat to combine masks and then apply any(axis=1)
     # Need to ensure indices are aligned before concatenation
     aligned_masks = [mask.reindex(df_processed.index, fill_value=False) for mask in interaction_top_30_masks]
     criterion_interaction_any_top_30 = pd.concat(aligned_masks, axis=1).any(axis=1)


# Combine the components of the second part of the OR condition: rating != 0.0 AND (any interaction term top 30%)
criterion_rating_not_zero_and_interaction = criterion_rating_not_zero & criterion_interaction_any_top_30

# Define the final 'Hit' variable based on the overall OR condition
df_processed['Hit'] = (criterion_rating_high | criterion_rating_not_zero_and_interaction).astype(int)

# Separate the DataFrame into Hit and Non-Hit samples based on the new definition
df_hit = df_processed[df_processed['Hit'] == 1].copy()
df_non_hit = df_processed[df_processed['Hit'] == 0].copy()

# Determine the number of Hit samples (minority class)
num_hit_samples = len(df_hit)

# Undersample the majority class (Non-Hit) to match the number of minority class samples (Hit)
# Ensure we don't sample more than the available number of Non-Hit samples
num_non_hit_to_keep = min(len(df_non_hit), num_hit_samples)
df_non_hit_undersampled = df_non_hit.sample(n=num_non_hit_to_keep, random_state=42)

# Combine the datasets (all Hit samples + undersampled Non-Hit samples)
df_balanced = pd.concat([df_hit, df_non_hit_undersampled])

# Shuffle the combined dataset to ensure randomness
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)


# Define features (X) and target variable (y) from the balanced DataFrame
# Drop the original 'id', 'name', 'released', and 'rating' columns as they are not features for the model
# Keep the interaction terms in X as they are now features
X = df_balanced.drop(['id', 'name', 'released', 'rating', 'Hit'], axis=1)
y = df_balanced['Hit']

# --- Fill NaN values in X with 0 ---
X.fillna(0, inplace=True)
print("\nFilled NaN values in X with 0.")


print(f"데이터 샘플링 후 데이터셋 개수 (행 수): {df_balanced.shape[0]}")
print("Features (X) shape:", X.shape)
print("Target variable (y) shape:", y.shape)
print(f"Hit 클래스 샘플 수: {y.sum()}")
print(f"Non-Hit 클래스 샘플 수: {len(y) - y.sum()}")

# Removed display(X.head()) and display(y.head()) from here


Filled NaN values in X with 0.
데이터 샘플링 후 데이터셋 개수 (행 수): 35018
Features (X) shape: (35018, 117)
Target variable (y) shape: (35018,)
Hit 클래스 샘플 수: 17509
Non-Hit 클래스 샘플 수: 17509


In [35]:
# Define a mapping for Russian tag column names to English
russian_to_english_tags = {
    'Автоматика': 'Automation',
    'Арена-шутер': 'Arena Shooter',
    'Викторина': 'Quiz',
    'Котики': 'Kittens', # Or Cats
    'Кулинария': 'Cooking',
    'Лутер-шутер': 'Looter Shooter',
    'Постапокалипсис': 'Post-apocalyptic',
    'Протагонистка': 'Female Protagonist',
    'Романтика': 'Romance',
    'Хентай': 'Hentai'
    # Add other Russian tags found in the data if necessary
}

# Rename the columns in the features DataFrame (X)
X.rename(columns=russian_to_english_tags, inplace=True)

# print("Columns in X after renaming:")
# display(X.columns.tolist())

# Add display of X.head() and y.head() here to show renamed columns
display(X.head())
display(y.head())

,playtime,Action,Adventure,Arcade,Board Games,Card,Casual,Educational,Family,Fighting,...,playtime_x_Fighting,playtime_x_Indie,playtime_x_Massively Multiplayer,playtime_x_Platformer,playtime_x_Puzzle,playtime_x_Racing,playtime_x_Shooter,playtime_x_Simulation,playtime_x_Sports,playtime_x_Strategy
0,-0.034983,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.0,-0.034983,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.034983,-0.034983,-0.0
1,0.603676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.603676,0.0
2,-0.034983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.0
3,-0.034983,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.034983,-0.000000,-0.000000,-0.0
4,-0.034983,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.0


,Hit
0,0
1,1
2,0
3,0
4,0


In [36]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (28014, 117)
Shape of X_test: (7004, 117)
Shape of y_train: (28014,)
Shape of y_test: (7004,)


In [37]:
import tensorflow as tf

# Check if a GPU is available and print its name
gpu_available = tf.config.list_physical_devices('GPU')

if gpu_available:
    print("GPU is available:")
    for gpu in gpu_available:
        print(f"  {gpu.name}")
else:
    print("No GPU available. Please check your Colab runtime settings (Runtime -> Change runtime type -> Hardware accelerator -> GPU).")

No GPU available. Please check your Colab runtime settings (Runtime -> Change runtime type -> Hardware accelerator -> GPU).


In [38]:
# Retrieve the scaled 'rating' column from the df_balanced DataFrame for regression target
y_reg = df_balanced['rating']

# Define features (X) for regression (X is already defined from df_balanced in cell 8ab43a4b)
# Ensure X is the same as used for classification after balancing and column renaming
# Let's confirm X is correctly set from df_balanced and has duplicate columns handled

# The X used for classification and regression should be the same after balancing and renaming
# X was already defined from df_balanced and duplicate columns were handled in cells 8ab43a4b and dbf026cc
# So, we just need to make sure y_reg aligns with this X.

# Display the first few rows and information of the features DataFrame X
print("Features DataFrame (X):")
display(X.head())
display(X.info())

# Display the first few rows and information of the new target variable y_reg
print("\nTarget Variable for Regression (y_reg):")
display(y_reg.head())
display(y_reg.info())

# Check if the number of samples in X and y_reg are consistent
if X.shape[0] != y_reg.shape[0]:
    print(f"\nError: Sample counts are inconsistent between X ({X.shape[0]}) and y_reg ({y_reg.shape[0]})")
else:
    print("\nSample counts are consistent between X and y_reg.")

Features DataFrame (X):


,playtime,Action,Adventure,Arcade,Board Games,Card,Casual,Educational,Family,Fighting,...,playtime_x_Fighting,playtime_x_Indie,playtime_x_Massively Multiplayer,playtime_x_Platformer,playtime_x_Puzzle,playtime_x_Racing,playtime_x_Shooter,playtime_x_Simulation,playtime_x_Sports,playtime_x_Strategy
0,-0.034983,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.0,-0.034983,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.034983,-0.034983,-0.0
1,0.603676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.603676,0.0
2,-0.034983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.0
3,-0.034983,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.034983,-0.000000,-0.000000,-0.0
4,-0.034983,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.000000,-0.000000,-0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35018 entries, 0 to 35017
Columns: 117 entries, playtime to playtime_x_Strategy
dtypes: float64(117)
memory usage: 31.3 MB


None


Target Variable for Regression (y_reg):


,rating
0,-0.141391
1,7.182621
2,-0.141391
3,-0.141391
4,-0.141391


<class 'pandas.core.series.Series'>
RangeIndex: 35018 entries, 0 to 35017
Series name: rating
Non-Null Count  Dtype  
--------------  -----  
35018 non-null  float64
dtypes: float64(1)
memory usage: 273.7 KB


None


Sample counts are consistent between X and y_reg.


In [39]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets for regression
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42, shuffle=True)

# Print the shapes of the resulting training and testing sets
print("Shape of X_train_reg:", X_train_reg.shape)
print("Shape of X_test_reg:", X_test_reg.shape)
print("Shape of y_train_reg:", y_train_reg.shape)
print("Shape of y_test_reg:", y_test_reg.shape)

Shape of X_train_reg: (28014, 117)
Shape of X_test_reg: (7004, 117)
Shape of y_train_reg: (28014,)
Shape of y_test_reg: (7004,)
